In [28]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import os
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [29]:
def preprocessing(text):
    
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace
    return ascii_text

In [30]:
"""
La fonction statut_extraction extrait les informations de statut de tabagisme, 
statut marital et statut d'alcool à partir d'un dictionnaire dico en parcourant les 
annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_tabac(doc):

    ## Initialisation
    statut = "UNKNOWN"

    # Nombre d'entités trouvées
    n_oui = 0
    n_non = 0

    # Proportions calculées à partir du nombre d'entités trouvées
    p_neg = 0
    p_pos = 0

    # Parcours des annotations pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "statut_tabagisme":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated == True:
                    n_non += 1
                # Sinon, on incrémente n_oui_tabac de 1
                else:
                    n_oui += 1

    if len(doc.anns) != 0:
        # Calcul des proportions de tabagisme
        p_neg = n_non / len(doc.anns)
        p_pos = n_oui / len(doc.anns)

    ## Choix du statut en fonction des proportions calculées
    if p_pos > p_neg:
        statut = "FUMEUR"
    elif p_pos < p_neg:
        statut = "NON-FUMEUR"


    return statut

In [31]:
"""
La fonction statut_extraction extrait les informations de statut d'alcool à partir d'un dictionnaire dico en
parcourant les  annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_alcool(doc):

    ## Initialisation
    statut = "UNKNOWN"

    # Nombre d'entités trouvées
    n_oui = 0
    n_non = 0

    # Proportions calculées à partir du nombre d'entités trouvées
    p_neg = 0
    p_pos = 0

    # Parcours des annotations pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "statut_alcool":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated == True:
                    n_non += 1
                    
                # Sinon, on incrémente n_oui_tabac de 1
                else:
                    n_oui += 1

    if len(doc.anns) != 0:
        # Calcul des proportions de tabagisme
        p_neg = n_non / len(doc.anns)
        p_pos = n_oui / len(doc.anns)
    ## Choix du statut en fonction des proportions calculées
    if p_pos > p_neg:
        statut = "ALCOOLIQUE"
    elif p_pos < p_neg:
        statut = "NON-AlCOOLIQUE"


    return statut


In [32]:
def statut_extraction_situation_familiale(doc):

    ## Initialisation
    statut_marital = "UNKNOWN"

    # On parcourt le dico pour analyser chaque annotation trouvé
    for ann in doc.anns:
        if ann.label== "statut_marital":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                
                statut_marital = ann.text.lower()
                
                 ## NORMALISATION: Seul, pas seul et inconnu
                if re.search(r"\bmarie[e]?\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\bcelibataire\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bdivorce[e]?\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bveuf\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bveuve\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bpacse[e][s]?\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\bconcubinage\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\b(vit|habite)\sseul(e)?\b", statut_marital):
                    statut_marital = "SEUL"
                    
                # Si il y a une négation
                if value_is_negated == True:
                    # On inverse statut_marital
                    if statut_marital == "SEUL":
                        statut_marital = "PAS SEUL"
                    else:
                        statut_marital = "SEUL"

    return statut_marital  

In [33]:
def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)
    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(717):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files_sorted[i]] = text
    return textes

## LES NEG DETECTOR

In [34]:

def neg_detector_tabac():

    ### QUELQUES REGEX NEGATION

    neg_rules = [       

    NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
    NegationDetectorRule(regexp=r"jamais"),
    NegationDetectorRule(regexp=r"arret"),
    NegationDetectorRule(regexp=r"ancien"),
    NegationDetectorRule(regexp=r"\bni\b"),
    NegationDetectorRule(regexp=r"\bnon\s+\b"),
    NegationDetectorRule(regexp=r"Tabac\s*[=:]?\s*0"),
    NegationDetectorRule(regexp=r"\bne/s*fume/s*pas\b"),
    NegationDetectorRule(regexp=r"tabagi(sm|qu)e\s+sevr"),
    NegationDetectorRule(regexp=r"(pas|ni|ou)\s+de\s+(consommation\s+de\s+)?taba"),
    NegationDetectorRule(regexp=r"pas\s+d\'intoxication\s+tabagi"),
    NegationDetectorRule(regexp=r"0 tabac"),

    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [35]:
def neg_detector_alcool():

    ### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
        NegationDetectorRule(regexp=r"\bni/s*alcool\b"),
        NegationDetectorRule(regexp=r"\bpas\b"),
        NegationDetectorRule(regexp=r"\becarte\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [36]:
def neg_detector_situation_familiale():

### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [37]:
# Listes pour stocker les données
data_tabac = []
data_alcool = []
data_familial = []

### QUELQUES REGEX RECHERCHE ENTITES
regexp_rules_tabac = [
    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"fumeur|fumeuse|paquet.{0,5}?ann[ée].?", label="statut_tabagisme", unicode_sensitive=True),
]

regexp_rules_alcool = [
    RegexpMatcherRule(regexp=r"\balcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bdependance\s*alcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="statut_alcool"),
]

regexp_rules_familial = [
    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="statut_marital"),
]

def extraction_finale(clinical_case_repo):
    # On charge les cas cliniques dans un dico {nom fichier: cas clinique}
    clinical_cases_dico = clinical_case_recovery(clinical_case_repo)

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case)
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)

        # Création de l'objet neg detector
        neg_detector_tabac_obj = neg_detector_tabac()
        neg_detector_alcool_obj = neg_detector_alcool()

        # On applique neg detector aux syntagmas
        neg_detector_tabac_obj.run(syntagmas)
        neg_detector_alcool_obj.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_familial = RegexpMatcher(rules=regexp_rules_familial, attrs_to_copy=["is_negated", "other_detected"])

        entities_tabac = regexp_matcher_tabac.run(syntagmas)
        entities_alcool = regexp_matcher_alcool.run(syntagmas)
        entities_familial = regexp_matcher_familial.run(syntagmas)

        for entity in entities_tabac:
            doc.anns.add((entity))
        for entity in entities_alcool:
            doc.anns.add((entity))
        for entity in entities_familial:
            doc.anns.add((entity))

        statut_tabagisme = statut_extraction_tabac(doc)
        statut_alcool = statut_extraction_alcool(doc)
        statut_familial = statut_extraction_situation_familiale(doc)

        # Remplissage de data
        data_tabac.append([fichier, clinical_case, statut_tabagisme])
        data_alcool.append([fichier, clinical_case, statut_alcool])
        data_familial.append([fichier, clinical_case, statut_familial])


    df_tabac = pd.DataFrame(data_tabac, columns=["nom fichier", "cas clinique", "statut tabagisme"])
    df_alcool = pd.DataFrame(data_alcool, columns=["nom fichier", "cas clinique", "statut alcool"])
    df_familial = pd.DataFrame(data_familial, columns=["nom fichier", "cas clinique", "statut familial"])

    #combined_df = pd.concat([df_tabac, df_alcool, df_familial], ignore_index=True)
    #combined_df = combined_df.sample(frac=1, random_state=42)
    
    combined_df = pd.concat([df_tabac, df_alcool, df_familial], axis=1)
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    combined_df = combined_df.dropna()
    
    return combined_df


In [38]:
# On crée un dataframe contenant les cas cliniques et les 3 statuts extraits
combined_df = extraction_finale("clinical_case2")

In [39]:
combined_df['statut alcool'].value_counts()

statut alcool
UNKNOWN           691
ALCOOLIQUE         20
NON-AlCOOLIQUE      6
Name: count, dtype: int64

In [40]:
combined_df.head(50)

nom fichier   
0      filehtml-1-cas.txt  \
1    filehtml-146-cas.txt   
2     filehtml-21-cas.txt   
3     filehtml-23-cas.txt   
4     filehtml-24-cas.txt   
5     filehtml-29-cas.txt   
6     filehtml-30-cas.txt   
7     filehtml-31-cas.txt   
8     filehtml-32-cas.txt   
9     filehtml-34-cas.txt   
10      filepdf-1-cas.txt   
11     filepdf-10-cas.txt   
12    filepdf-100-cas.txt   
13    filepdf-101-cas.txt   
14    filepdf-102-cas.txt   
15  filepdf-103-1-cas.txt   
16  filepdf-103-2-cas.txt   
17  filepdf-103-3-cas.txt   
18    filepdf-104-cas.txt   
19    filepdf-105-cas.txt   
20    filepdf-106-cas.txt   
21    filepdf-107-cas.txt   
22    filepdf-108-cas.txt   
23    filepdf-109-cas.txt   
24     filepdf-11-cas.txt   
25  filepdf-110-1-cas.txt   
26  filepdf-110-2-cas.txt   
27    filepdf-111-cas.txt   
28    filepdf-112-cas.txt   
29    filepdf-113-cas.txt   
30    filepdf-114-cas.txt   
31    filepdf-115-cas.txt   
32    filepdf-116-cas.txt   
33    filepdf-117-cas.txt   
34    filepdf-118-cas.txt   
35    filepdf-119-cas.txt   
36     filepdf-12-cas.txt   
37    filepdf-120-cas.txt   
38    filepdf-121-cas.txt   
39    filepdf-122-cas.txt   
40    filepdf-123-cas.txt   
41    filepdf-124-cas.txt   
42    filepdf-125-cas.txt   
43    filepdf-126-cas.txt   
44    filepdf-127-cas.txt   
45    filepdf-128-cas.txt   
46    filepdf-129-cas.txt   
47   filepdf-13-1-cas.txt   
48   filepdf-13-2-cas.txt   
49    filepdf-130-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# PREPARATION DES FICHIERS A ANNOTER 

## PREPARATION ANNOTATION TABAC

In [41]:
combined_df['statut tabagisme'].value_counts()

statut tabagisme
UNKNOWN       664
FUMEUR         45
NON-FUMEUR      8
Name: count, dtype: int64

In [42]:
df_non_fumeur = combined_df[combined_df["statut tabagisme"] == "NON-FUMEUR"]
df_fumeur = combined_df[combined_df["statut tabagisme"] == "FUMEUR"].head(20)
df_unknown = combined_df[combined_df["statut tabagisme"] == "UNKNOWN"].head(22)

In [43]:
df_statut_tabac = pd.concat([df_non_fumeur, df_fumeur, df_unknown], ignore_index=True)

In [44]:
df_statut_tabac.shape

(50, 5)

In [45]:
df_statut_tabac.head(50)

nom fichier   
0     filehtml-24-cas.txt  \
1     filehtml-31-cas.txt   
2     filepdf-138-cas.txt   
3      filepdf-69-cas.txt   
4     filepdf-750-cas.txt   
5     filepdf-756-cas.txt   
6     filepdf-757-cas.txt   
7   filepdf-883-1-cas.txt   
8      filehtml-1-cas.txt   
9   filepdf-103-1-cas.txt   
10    filepdf-130-cas.txt   
11     filepdf-16-cas.txt   
12  filepdf-171-4-cas.txt   
13    filepdf-193-cas.txt   
14    filepdf-205-cas.txt   
15    filepdf-221-cas.txt   
16    filepdf-223-cas.txt   
17    filepdf-239-cas.txt   
18  filepdf-263-1-cas.txt   
19  filepdf-263-2-cas.txt   
20  filepdf-263-3-cas.txt   
21    filepdf-272-cas.txt   
22    filepdf-273-cas.txt   
23     filepdf-28-cas.txt   
24     filepdf-29-cas.txt   
25    filepdf-454-cas.txt   
26    filepdf-474-cas.txt   
27     filepdf-50-cas.txt   
28   filehtml-146-cas.txt   
29    filehtml-21-cas.txt   
30    filehtml-23-cas.txt   
31    filehtml-29-cas.txt   
32    filehtml-30-cas.txt   
33    filehtml-32-cas.txt   
34    filehtml-34-cas.txt   
35      filepdf-1-cas.txt   
36     filepdf-10-cas.txt   
37    filepdf-100-cas.txt   
38    filepdf-101-cas.txt   
39    filepdf-102-cas.txt   
40  filepdf-103-2-cas.txt   
41  filepdf-103-3-cas.txt   
42    filepdf-104-cas.txt   
43    filepdf-105-cas.txt   
44    filepdf-106-cas.txt   
45    filepdf-107-cas.txt   
46    filepdf-108-cas.txt   
47    filepdf-109-cas.txt   
48     filepdf-11-cas.txt   
49  filepdf-110-1-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [46]:
df_statut_tabac['statut tabagisme'].value_counts()

statut tabagisme
UNKNOWN       22
FUMEUR        20
NON-FUMEUR     8
Name: count, dtype: int64

In [49]:
# Récupération dans un dossier des fichiers textes à annoter

import os
import shutil

chemin_dossier_source = "clinical_case2"
chemin_dossier_destination = "50_fichiers_annotation_tabac"
# On crée le dossier destination si il n'existe pas 
os.makedirs(chemin_dossier_destination, exist_ok=True)

for nom_fichier in df_statut_tabac['nom fichier']:
    chemin_fichier_source = os.path.join(chemin_dossier_source, nom_fichier)
    chemin_fichier_destination = os.path.join(chemin_dossier_destination, nom_fichier)
    shutil.copy(chemin_fichier_source, chemin_fichier_destination)


## PREPARATION ANNOTATION ALCOOL

In [19]:
df_non_alcool = combined_df[combined_df["statut alcool"] == "NON-AlCOOLIQUE"]
df_alcool = combined_df[combined_df["statut alcool"] == "ALCOOLIQUE"].head(20)
df_unknown = combined_df[combined_df["statut alcool"] == "UNKNOWN"].head(24)

In [20]:
df_statut_alcool = pd.concat([df_non_alcool, df_alcool, df_unknown], ignore_index=True)

In [21]:
df_statut_alcool['statut alcool'].value_counts()

statut alcool
UNKNOWN           24
ALCOOLIQUE        20
NON-AlCOOLIQUE     6
Name: count, dtype: int64

In [22]:
df_statut_alcool.shape

(50, 5)

## PREPARATION ANNOTATION SITUATION FAMILIALE

In [23]:
combined_df['statut familial'].value_counts()

statut familial
UNKNOWN     696
SEUL         11
PAS SEUL     10
Name: count, dtype: int64

In [24]:
df_seul = combined_df[combined_df["statut familial"] == "SEUL"]
df_pas_seul = combined_df[combined_df["statut familial"] == "PAS SEUL"]
df_unknown = combined_df[combined_df["statut familial"] == "UNKNOWN"].head(29)

In [25]:
df_statut_familial = pd.concat([df_seul, df_pas_seul, df_unknown], ignore_index=True)

In [26]:
df_statut_familial['statut familial'].value_counts()

statut familial
UNKNOWN     29
SEUL        11
PAS SEUL    10
Name: count, dtype: int64

In [27]:
df_statut_familial.shape

(50, 5)